In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from CHAID import Tree

In [3]:
data = pd.read_csv("tr_code.csv")
data.drop(data.columns[0],axis=1,inplace=True)
data = data[~data['familyGroup'].isin(['NON_PRODUCT','BREAKFAST_DRINK','GIFT_COUPON'])]

In [5]:
quantity_by_individualItems = data.pivot_table(values=['qty'], index = data.key, columns='ItemName', aggfunc='sum').fillna(0)
quantity_by_individualItems.columns = list(map("_".join, quantity_by_individualItems.columns))
final_dataset = quantity_by_individualItems \
.join(data[["key","Channel","saleType","timeOfDay","receiptToPresentTime"]].set_index("key"),on=['key'],how="left").reset_index()

In [30]:
df = final_dataset[final_dataset["Channel"]=="Front Counter"].drop(["saleType","timeOfDay","Channel","key"], axis = 1)
df = df[df['receiptToPresentTime']!='[nan nan]']
df['receiptToPresentTime'] = df['receiptToPresentTime'].astype('float').map(lambda x : round(x,2))

In [32]:
## set the CHAID input parameters
independent_variable_columns = list(df.drop(["receiptToPresentTime"],axis=1).columns)
dep_variable = 'receiptToPresentTime'

In [24]:
## create the Tree via pandas
#tree = Tree.from_pandas_df(df, dict(zip(independent_variable_columns, ['nominal'] * 3)), dep_variable)

In [25]:
## create the same tree, but without pandas helper
#tree = Tree.from_numpy(ndarr, arr, split_titles=['a', 'b', 'c'], min_child_node_size=5)

In [26]:
## create the same tree using the tree constructor
# cols = [
#   NominalColumn(ndarr[:,0], name='a')
#   NominalColumn(ndarr[:,1], name='b')
#   NominalColumn(ndarr[:,2], name='c')
# ]
# tree = Tree(cols, NominalColumn(arr, name='d'), {'min_child_node_size': 5})

In [28]:
#tree.print_tree()

In [33]:
tree = Tree.from_pandas_df(df, dict(zip(independent_variable_columns, ['nominal'] * 3)), dep_variable, dep_variable_type='continuous')

In [34]:
tree.print_tree()

([], {'mean': 212.49394166666664, 's.t.d': 226.8619830162088}, <Invalid Chaid Split> - splitting would create nodes with less than the minimum child node size)



In [35]:
tree.to_tree()

In [36]:
first_node = tree.tree_store[0]

In [38]:
first_node

([], {'mean': 212.49394166666664, 's.t.d': 226.8619830162088}, <Invalid Chaid Split> - splitting would create nodes with less than the minimum child node size)

In [39]:
first_node.members

{'mean': 212.49394166666664, 's.t.d': 226.8619830162088}